In [ ]:
import numpy as np
import pandas as pd

import xarray as xr

import dask
dask.config.set( { "scheduler": "threads" } )

In [ ]:
# Constantes
Rw  = 461.4 
Ra  = 286.9
Lv  = 2501
cpa = 1.006 
cpv = 1.86

# Presión de vapor de saturación.
def Pvs( T ): # Pa
    return np.exp( 77.3450 + 0.0057 * T - 7235 / T ) / T**8.2

# Humedad absoluta de saturación.
def rs( T, P ): # g/kg
    return ( 1000 * (Ra/Rw) * Pvs(T) ) / ( P - Pvs(T) )

In [ ]:
path = "/Users/rodrigo/Desktop/zzR_zz_Mega_Variables_Extraidas.nc"#
path_2 = "/Users/rodrigo/Desktop/zzR_zz_Mega_Variables_Extraidas_2.nc"

ds = xr.open_dataset( path, chunks = {"south_north": 60, "west_east": 70} )
ds = ds.rename_vars( { "XTIME": "time", "XLONG": "lon", "XLAT": "lat",
    "T2": "Temperature", "PSFC": "Pressure", "SWDOWN": "GHI" } )
ds["lon"] = ds["lon"].isel({"south_north": 0})
ds["lat"] = ds["lat"].isel({"west_east": 0})
ds = ds.swap_dims({"south_north": "lat", "west_east": "lon", "XTIME": "time"})

ds["Wind Speed"] = np.sqrt( np.square(ds["U10"]) + np.square(ds["V10"]) )

ds["Wind Direction"] = np.arctan2( ds["V10"], ds["U10"] ) * 180 / np.pi - 90
ds["Wind Direction"] = ds["Wind Direction"].where(
    ds["Wind Direction"] > 0, ds["Wind Direction"] + 360 )

ds["Relative Humidity"] = ( 100 * ds["Q2"]
    / rs( ds["Temperature"], ds["Pressure"] ) )
ds["Relative Humidity"] = ds["Relative Humidity"].where(
    ds["Relative Humidity"] > 100, 100 )

ds = ds.drop_vars( ["U10", "V10", "Q2"] )

ds["Wind Speed"] = ds["Wind Speed"].assign_attrs( units = "m/s" )
ds["Wind Direction"] = ds["Wind Direction"].assign_attrs( units = "degrees" )
ds["Relative Humidity"] = ds["Relative Humidity"].assign_attrs( units = "%" )

ds.to_netcdf( path_2 )

ds